In [1]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np

from collections import defaultdict
from tqdm import tqdm 
from heapq import nlargest

tqdm.pandas()

In [20]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [21]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [22]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 469568/469568 [00:12<00:00, 38210.77it/s]


In [23]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [24]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]
len(career_paths)

113724

In [32]:
career_paths_train = career_paths.iloc[:int(0.8 * 113724)]
career_paths_test = career_paths.iloc[int(0.8 * 113724):]

In [33]:
switches = defaultdict(lambda: defaultdict(int))

for c, jobs in career_paths_train.iteritems():
    for i in range(len(jobs)):
        if i < len(jobs) - 1 and jobs[i] != jobs[i + 1]:
            switches[jobs[i]][jobs[i  + 1]] += 1

In [44]:
most_common = pd.Series({k: set(nlargest(10, v, key=v.get)) for k, v in switches.items()})

In [45]:
most_common.head()

208    {185, 229, 335, 177, 209, 344, 345, 348, 189, ...
348    {195, 229, 207, 208, 335, 177, 344, 345, 189, ...
344    {323, 195, 185, 332, 335, 208, 177, 117, 343, ...
345    {323, 195, 326, 199, 332, 335, 208, 177, 343, ...
185    {193, 195, 227, 174, 208, 177, 344, 345, 188, ...
dtype: object

In [46]:
most_common_switch = career_paths_test.progress_apply(lambda x: x[-1] in most_common.loc[x[-2]] if x[-2] in most_common.index else False).mean()

100%|██████████| 22745/22745 [00:00<00:00, 46453.27it/s]


In [47]:
most_common_switch, 1.96 * np.sqrt(((1 - most_common_switch)* (most_common_switch)) / (113724*0.2))

(0.6130138491976258, 0.006329914804694123)

In [48]:
print(f"Most common switch accuracy: {most_common_switch * 100:>.2f}%")

Most common switch accuracy: 61.30%
